# VQD (Noiseless) Comparisons

Pure package client for comparing VQD performance across:

1) Optimizers (fixed ansatz; optimizer-specific step sizes)
2) Ansatzes   (fixed optimizer)
3) Full grid  (all ansatz × optimizer combos)
4) Pick a "best" config using an explicit excited-state target policy
5) Validate top configs across multiple seeds (mean ± std)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Sequence, Tuple

import numpy as np

from vqe.core import get_exact_spectrum
from vqe.vqd import run_vqd

## Configuration

In [ ]:
molecule = "H2"
mapping = "jordan_wigner"
unit = "angstrom"

num_states = 2  # ground + first excited
noisy = False

# Only used if noisy=True
depolarizing_prob = 0.0
amplitude_damping_prob = 0.0
noise_model = None

# VQD deflation schedule
beta = 10.0
beta_start = 0.0
beta_ramp = "cosine"
beta_hold_fraction = 0.1

# Optimization budget
steps = 250
seed = 0

# Optimizer sweep
optimizers = ["Adam", "GradientDescent", "Momentum", "Nesterov", "RMSProp", "Adagrad"]
stepsize_map = {
    "Adam": 0.2,
    "GradientDescent": 0.05,
    "Momentum": 0.1,
    "Nesterov": 0.1,
    "RMSProp": 0.1,
    "Adagrad": 0.2,
}
fixed_ansatz = "UCCSD"

# Ansatz sweep
ansatzes = [
    "UCC-S",
    "UCC-D",
    "UCCSD",
    "Minimal",
    "RY-CZ",
    "TwoQubit-RY-CNOT",
    "StronglyEntanglingLayers",
]
fixed_optimizer = "Adam"
fixed_stepsize = float(stepsize_map[fixed_optimizer])

# "Best config" target policy
excited_target_mode = "first"  # "first" or "low_k"
excited_target_k = 6           # used only if mode="low_k"

## Exact spectrum benchmark (package-provided)

In [ ]:
evals = np.asarray(
    get_exact_spectrum(molecule=molecule, k=10, mapping=mapping, unit=unit),
    dtype=float,
)

print(f"Molecule: {molecule}")
print(f"Mapping:  {mapping}")
print("Exact lowest eigenvalues (Ha):")
for i, e in enumerate(evals):
    print(f"#{i:>2}: {float(e): .10f}")

## Run wrapper + table printer

In [ ]:
@dataclass(frozen=True)
class VQDRow:
    ansatz: str
    optimizer: str
    stepsize: float
    E_final: Tuple[float, ...]  # length = num_states


def _run_vqd_final_energies(
    *,
    ansatz_name: str,
    optimizer_name: str,
    stepsize: float,
    seed: int,
) -> Tuple[float, ...]:
    out = run_vqd(
        molecule=molecule,
        num_states=num_states,
        beta=beta,
        beta_start=beta_start,
        beta_ramp=beta_ramp,
        beta_hold_fraction=beta_hold_fraction,
        ansatz_name=ansatz_name,
        optimizer_name=optimizer_name,
        steps=steps,
        stepsize=float(stepsize),
        seed=int(seed),
        noisy=noisy,
        depolarizing_prob=float(depolarizing_prob),
        amplitude_damping_prob=float(amplitude_damping_prob),
        noise_model=noise_model,
        plot=False,
        force=False,
        mapping=mapping,
    )
    energies_per_state = out["energies_per_state"]
    return tuple(float(traj[-1]) for traj in energies_per_state)


def _print_rows(rows: Sequence[VQDRow], *, title: str) -> None:
    if not rows:
        print(f"\n{title}\n(no rows)\n")
        return

    energy_cols = [f"E{i}" for i in range(num_states)]
    headers = ["Ansatz", "Optimizer", "Stepsize"] + [f"{c} (Ha)" for c in energy_cols]

    def fmt_row(r: VQDRow) -> List[str]:
        parts = [r.ansatz, r.optimizer, f"{r.stepsize:g}"]
        parts += [f"{e:+.10f}" for e in r.E_final]
        return parts

    data = [fmt_row(r) for r in rows]
    widths = [len(h) for h in headers]
    for row in data:
        widths = [max(w, len(cell)) for w, cell in zip(widths, row)]

    print("\n" + title)
    line = "  ".join(h.ljust(w) for h, w in zip(headers, widths))
    print(line)
    print("-" * len(line))
    for row in data:
        print("  ".join(cell.ljust(w) for cell, w in zip(row, widths)))

## 1) Optimizer comparison (fixed ansatz)

In [ ]:
optimizer_rows: List[VQDRow] = []

for opt in optimizers:
    ss = float(stepsize_map[opt])
    print(f"Running: ansatz={fixed_ansatz}, optimizer={opt}, stepsize={ss} ...")
    finals = _run_vqd_final_energies(
        ansatz_name=fixed_ansatz,
        optimizer_name=opt,
        stepsize=ss,
        seed=int(seed),
    )
    optimizer_rows.append(VQDRow(fixed_ansatz, opt, ss, finals))

optimizer_rows_sorted = sorted(
    optimizer_rows, key=lambda r: r.E_final[1] if num_states > 1 else r.E_final[0]
)
_print_rows(optimizer_rows_sorted, title=f"VQD Optimizer Comparison (ansatz={fixed_ansatz})")

## 2) Ansatz comparison (fixed optimizer)

In [ ]:
ansatz_rows: List[VQDRow] = []

for ans in ansatzes:
    print(f"Running: ansatz={ans}, optimizer={fixed_optimizer}, stepsize={fixed_stepsize} ...")
    finals = _run_vqd_final_energies(
        ansatz_name=ans,
        optimizer_name=fixed_optimizer,
        stepsize=fixed_stepsize,
        seed=int(seed),
    )
    ansatz_rows.append(VQDRow(ans, fixed_optimizer, fixed_stepsize, finals))

ansatz_rows_sorted = sorted(
    ansatz_rows, key=lambda r: r.E_final[1] if num_states > 1 else r.E_final[0]
)
_print_rows(ansatz_rows_sorted, title=f"VQD Ansatz Comparison (optimizer={fixed_optimizer})")

## 3) Full grid (all ansatz × optimizer combos)

In [ ]:
grid_rows: List[VQDRow] = []

for ans in ansatzes:
    for opt in optimizers:
        ss = float(stepsize_map[opt])
        print(f"Running: ansatz={ans}, optimizer={opt}, stepsize={ss} ...")
        finals = _run_vqd_final_energies(
            ansatz_name=ans,
            optimizer_name=opt,
            stepsize=ss,
            seed=int(seed),
        )
        grid_rows.append(VQDRow(ans, opt, ss, finals))

grid_rows_sorted = sorted(
    grid_rows,
    key=lambda r: (r.E_final[1] if num_states > 1 else r.E_final[0], r.E_final[0]),
)
_print_rows(grid_rows_sorted, title="VQD Full Grid (all ansatz × optimizer combos)")

## 4) Choose the "best" configuration

Definitions:
- ΔE0 = |E0 - exact[0]|
- ΔE1 depends on `excited_target_mode`:
    - "first": |E1 - exact[1]|
    - "low_k": min_{1≤j≤k} |E1 - exact[j]|

Score: ΔE0 + ΔE1

In [ ]:
def _excited_error(E1: float, exact: np.ndarray) -> float:
    if len(exact) < 2:
        return float("nan")

    mode = str(excited_target_mode).strip().lower()
    if mode == "first":
        return float(abs(E1 - float(exact[1])))

    if mode == "low_k":
        k = max(1, int(excited_target_k))
        tgt = exact[1 : 1 + k]
        return float(np.min(np.abs(tgt - E1)))

    raise ValueError("excited_target_mode must be 'first' or 'low_k'.")


def _score_row(r: VQDRow, exact: np.ndarray) -> Dict[str, Any]:
    E0 = float(r.E_final[0])
    E1 = float(r.E_final[1]) if len(r.E_final) > 1 else float("nan")
    dE0 = abs(E0 - float(exact[0]))
    dE1 = _excited_error(E1, exact)
    return {"row": r, "E0": E0, "E1": E1, "dE0": dE0, "dE1": dE1, "score": dE0 + dE1}


scored = [_score_row(r, evals) for r in grid_rows]
scored_sorted = sorted(scored, key=lambda x: (x["score"], x["dE1"], x["dE0"]))

best = scored_sorted[0]
rbest: VQDRow = best["row"]

print(f"\nTarget policy: excited_target_mode={excited_target_mode!r}, excited_target_k={excited_target_k}")
print("\nBest configuration:")
print(f"  Ansatz:    {rbest.ansatz}")
print(f"  Optimizer: {rbest.optimizer}")
print(f"  Stepsize:  {rbest.stepsize:g}")
print(f"  E0_final:  {best['E0']:+.10f}   (ΔE0={best['dE0']:.3e} Ha)")
print(f"  E1_final:  {best['E1']:+.10f}   (ΔE1={best['dE1']:.3e} Ha)")
print(f"  Score:     {best['score']:.3e}")

top_k = 10
print(f"\nTop {top_k} configurations:")
for i, x in enumerate(scored_sorted[:top_k], start=1):
    rr: VQDRow = x["row"]
    print(
        f"{i:>2}. ansatz={rr.ansatz:<22} optimizer={rr.optimizer:<14} stepsize={rr.stepsize:<5g}  "
        f"E0={x['E0']:+.10f} (ΔE0={x['dE0']:.2e})  "
        f"E1={x['E1']:+.10f} (ΔE1={x['dE1']:.2e})  "
        f"score={x['score']:.2e}"
    )

## 5) Validate top configurations across multiple seeds (mean ± std)

In [ ]:
seeds = np.arange(5)
top_k_validate = 5
top_configs = scored_sorted[:top_k_validate]

def _mean_std(xs: Sequence[float]) -> Tuple[float, float]:
    a = np.asarray(xs, dtype=float)
    if len(a) <= 1:
        return float(a.mean()), 0.0
    return float(a.mean()), float(a.std(ddof=1))


val_rows: List[Dict[str, Any]] = []

for i, x in enumerate(top_configs, start=1):
    rr: VQDRow = x["row"]

    E0s: List[float] = []
    E1s: List[float] = []
    dE0s: List[float] = []
    dE1s: List[float] = []

    print(f"\nValidating {i}/{len(top_configs)}: ansatz={rr.ansatz}, optimizer={rr.optimizer}, stepsize={rr.stepsize:g}, seeds={seeds}")

    for s in seeds:
        finals = _run_vqd_final_energies(
            ansatz_name=rr.ansatz,
            optimizer_name=rr.optimizer,
            stepsize=float(rr.stepsize),
            seed=int(s),
        )
        E0 = float(finals[0])
        E1 = float(finals[1]) if len(finals) > 1 else float("nan")

        E0s.append(E0)
        E1s.append(E1)
        dE0s.append(abs(E0 - float(evals[0])))
        dE1s.append(_excited_error(E1, evals))

    E0_mean, E0_std = _mean_std(E0s)
    E1_mean, E1_std = _mean_std(E1s)
    dE0_mean, dE0_std = _mean_std(dE0s)
    dE1_mean, dE1_std = _mean_std(dE1s)

    val_rows.append(
        {
            "ansatz": rr.ansatz,
            "optimizer": rr.optimizer,
            "stepsize": rr.stepsize,
            "E0_mean": E0_mean,
            "E0_std": E0_std,
            "E1_mean": E1_mean,
            "E1_std": E1_std,
            "dE0_mean": dE0_mean,
            "dE0_std": dE0_std,
            "dE1_mean": dE1_mean,
            "dE1_std": dE1_std,
            "score_mean": dE0_mean + dE1_mean,
        }
    )

val_rows_sorted = sorted(val_rows, key=lambda r: (r["score_mean"], r["dE1_std"], r["dE0_std"]))

print("\nMulti-seed validation summary (sorted by mean score):")
hdr = (
    f"{'Ansatz':<22}  {'Optimizer':<14}  {'SS':<5}  "
    f"{'E0 mean±std':<26}  {'E1 mean±std':<26}  "
    f"{'ΔE0 mean±std':<26}  {'ΔE1 mean±std':<26}  {'score_mean':<10}"
)
print(hdr)
print("-" * len(hdr))

for r in val_rows_sorted:
    print(
        f"{r['ansatz']:<22}  {r['optimizer']:<14}  {r['stepsize']:<5g}  "
        f"{r['E0_mean']:+.10f}±{r['E0_std']:.2e}  "
        f"{r['E1_mean']:+.10f}±{r['E1_std']:.2e}  "
        f"{r['dE0_mean']:.2e}±{r['dE0_std']:.2e}  "
        f"{r['dE1_mean']:.2e}±{r['dE1_std']:.2e}  "
        f"{r['score_mean']:.2e}"
    )

best_mean = val_rows_sorted[0]
print("\nBest configuration by mean score across seeds:")
print(f"  Ansatz:    {best_mean['ansatz']}")
print(f"  Optimizer: {best_mean['optimizer']}")
print(f"  Stepsize:  {best_mean['stepsize']}")
print(f"  E0:        {best_mean['E0_mean']:+.10f} ± {best_mean['E0_std']:.2e}")
print(f"  E1:        {best_mean['E1_mean']:+.10f} ± {best_mean['E1_std']:.2e}")
print(f"  ΔE0:       {best_mean['dE0_mean']:.2e} ± {best_mean['dE0_std']:.2e}")
print(f"  ΔE1:       {best_mean['dE1_mean']:.2e} ± {best_mean['dE1_std']:.2e}")
print(f"  score:     {best_mean['score_mean']:.2e}")